In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Get dataframe
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/CSC413FinalProject/liar_dataset/train.tsv', delimiter='\t', header=None)
# Optionally, add column names if the file doesn't include headers
df.columns = ["ID", "Label", "Statement", "Subject", "Speaker", "Speaker_Job", "Speaker_State", "Party", "barely_true_counts", "false_counts", "half_true_counts", "mostly_true_counts", "pants_on_fire_counts", "context"]

In [3]:
# Get Statements
statements = list(df['Statement'])

In [98]:
# Get dataframe
import pandas as pd
df_val = pd.read_csv('/content/drive/MyDrive/CSC413FinalProject/liar_dataset/valid.tsv', delimiter='\t', header=None)
# Optionally, add column names if the file doesn't include headers
df_val.columns = ["ID", "Label", "Statement", "Subject", "Speaker", "Speaker_Job", "Speaker_State", "Party", "barely_true_counts", "false_counts", "half_true_counts", "mostly_true_counts", "pants_on_fire_counts", "context"]

In [99]:
statements_val = list(df_val['Statement'])

In [108]:
statements_val[0]
statements[0]

'Says the Annies List political group supports third-trimester abortions on demand.'

In [4]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [102]:
# Tokenize and convert statements to BERT input format
inputs = tokenizer(statements_val, padding=True, truncation=True, return_tensors="pt")

In [103]:
# Pass inputs through BERT model to get embeddings
with torch.no_grad():
    outputs = bert_model(**inputs)

In [104]:
# Get the hidden states from the BERT model
last_hidden_states = outputs.last_hidden_state

In [105]:
last_hidden_states.shape

torch.Size([1284, 77, 768])

In [106]:
torch.save(last_hidden_states, '/content/drive/MyDrive/CSC413FinalProject/RNN_embeddings_val.pt')

In [93]:
names = ['final']
train_embeddings = torch.load('/content/drive/MyDrive/CSC413FinalProject/RNN_embeddings.pt')
for i in names:
  embed = torch.load('RNN_inputs_'+i+'.pt')
  train_embeddings = torch.cat((train_embeddings, embed), dim=0)

In [94]:
train_embeddings.shape


torch.Size([10240, 512, 768])

In [95]:
torch.save(train_embeddings, '/content/drive/MyDrive/CSC413FinalProject/RNN_embeddings.pt')

In [ ]:
# mean pooling
sentence_embeddings = torch.mean(last_hidden_states, dim=1)

In [28]:
# Define RNN model
class RNNClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(RNNClassifier, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        _, hn = self.rnn(x)
        hn = hn.squeeze(0)
        out = self.fc(hn)  # Take the last hidden state of the sequence
        return out

In [37]:
# Get Training labels
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np

labels = list(df['Label'])[1000:2000]

# Example list
categories = np.array(labels).reshape(-1, 1)

# Create the encoder and fit it
encoder = OneHotEncoder(sparse=False)
labels = encoder.fit_transform(categories)

print(labels.shape)

(1000, 6)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [30]:
# Prepare data loader
from torch.utils.data import DataLoader, TensorDataset
dataset = TensorDataset(last_hidden_states, torch.tensor(labels))
data_loader = DataLoader(dataset, batch_size=2, shuffle=True)

In [32]:
#parameters for the RNN model
input_size = last_hidden_states.shape[-1]  # Size of BERT hidden states
hidden_size = 32
num_classes = 6

# Initialize RNN model
rnn_model = RNNClassifier(input_size, hidden_size, num_classes)

In [33]:
# Training loop
import torch.optim as optim
optimizer = optim.Adam(rnn_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
num_epochs = 5
for epoch in range(num_epochs):
    total_loss = 0
    for inputs, targets in data_loader:
        optimizer.zero_grad()
        outputs = rnn_model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss}")

Epoch 1, Loss: 894.5072713494301
Epoch 2, Loss: 854.9571096301079
Epoch 3, Loss: 814.6833645701408
Epoch 4, Loss: 778.8588905185461
Epoch 5, Loss: 726.147174179554


In [38]:
#Train Accuracy
model_outputs = torch.argmax(rnn_model(last_hidden_states.to('cpu')), axis =1)
print(labels.shape)
argmax_labels = torch.argmax(torch.Tensor(labels), axis =1)
print(model_outputs.shape)
print(argmax_labels.shape)

print(sum(model_outputs == argmax_labels)/len(model_outputs))

(1000, 6)
torch.Size([1000])
torch.Size([1000])
tensor(0.5420)
